Import Libraries

In [ ]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
from flax import nnx
import optax
import matplotlib.pyplot as plt
from tqdm import tqdm

Unpickling the data

In [63]:
# Due to errors I was experiencing this seems to be the quickest fix I could find to allow me to unpickle the data
import sys
import types
import pickle

fake_module = types.ModuleType("DataSetup")

class DataStore:
    def __init__(self):
        pass

fake_module.DataStore = DataStore

sys.modules["DataSetup"] = fake_module

data_file_1 = r"C:\Users\samue\Downloads\Simulation.pickle"
data_file_2 = r"C:\Users\samue\Downloads\Simulation 2.pickle"

with open(data_file_1,"rb") as f:
    data_unpickled_1 = pickle.load(f)

with open(data_file_2,"rb") as f:
    data_unpickled_2 = pickle.load(f)

_,data_object_1 = data_unpickled_1
_,data_object_2 = data_unpickled_2

input_dataset_1 = jnp.array(data_object_1.Indata)
#data_index_1 = data_object_1.i
e_dataset_1 = jnp.array(data_object_1.SE)
e_prime_dataset_1 = jnp.array(data_object_1.Jac)

input_dataset_2 = jnp.array(data_object_2.Indata)
#data_index_2 = data_object_2.i
e_dataset_2 = jnp.array(data_object_2.SE)
e_prime_dataset_2 = jnp.array(data_object_2.Jac)

print(input_dataset_2.shape)
print(input_dataset_1.shape)
print(e_dataset_1.shape)
print(e_prime_dataset_1.shape)

input_dataset = jax.numpy.concatenate([input_dataset_1,input_dataset_2],axis=0)
target_e_dataset = jax.numpy.concatenate([e_dataset_1, e_dataset_2],axis=0)
target_e_dataset = jax.numpy.expand_dims(target_e_dataset,axis=1)
target_e_prime_dataset = jax.numpy.concatenate([e_prime_dataset_1,e_prime_dataset_2],axis=0)
print(input_dataset.shape)
print(target_e_dataset.shape)
print(target_e_prime_dataset.shape)

(10000, 152, 3)
(10000, 152, 3)
(10000,)
(10000, 152, 3)
(20000, 152, 3)
(20000, 1)
(20000, 152, 3)


Redimensionalise

In [ ]:
def Redimensionalise(self):
    self.Disp = np.zeros((self.Dims,self.Dims,self.Dims,3))
    m = 0
    for i in range(self.Dims):
        for j in range(self.Dims):
            for k in range(self.Dims):
                if self.xInMesh[0][i,j,k] == 0 or self.xInMesh[0][i,j,k] == 1 or self.xInMesh[1][i,j,k] == 0 or self.xInMesh[1][i,j,k] == 1 or self.xInMesh[2][i,j,k] == 0 or self.xInMesh[2][i,j,k] == 1:
                    self.Disp[i,j,k,:] = self.RandDisp[self.Index,m,:]
                    m = m +1
    return self.Disp
    

Create Batches function

In [ ]:
def get_batches(dataset, batch_size, shuffle=True):
    """
    Creates data batches for training
    """
    N = dataset['displacements'].shape[0]
    indices = jnp.arange(N)
    if shuffle:
        indices = jax.random.permutation(jax.random.PRNGKey(0), indices)
    
    for start in range(0, N, batch_size):
        end = start + batch_size
        batch_idx = indices[start:end]
        
        yield {
            key: value[batch_idx]
            for key, value in dataset.items()
        }

RNG key

In [ ]:
seed = 42 # This can be changed but is here to make the results easy to reproduce

base_key = jax.random.PRNGkey(seed)
rngs = nnx.Rngs(base_key)

Hyper Params

In [57]:
Epochs = 1000
alpha = 1.0
gamma = 1.0
lambda_ = 1.0
Learn_Rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
Batch_size = 10

Dataset

In [64]:
batch_num = input_dataset.shape[0] // Batch_size

# input_dataset target_e_dataset target_e_prime_dataset 

Dataset = {
    'displacements':input_dataset,
    'target_e':target_e_dataset,
    'target_e_prime':target_e_prime_dataset
}



Node Classes and Acivations

In [ ]:
class Linear(nnx.module):
    """Linear node for neural network"""

    def __init__(self,din: int,dout: int,*,rngs: nnx.Rngs):
        key = rngs.params()
        self.W = nnx.Param(jax.random.uniform(key=key, shape=(din,dout)))
        self.b = nnx.Param(jnp.zeros(shape=(dout,)))
        self.din, self.dout = din, dout

    def __call__(self,x: jax.Array):
        return(x @ self.W + self.b)
    
def SiLU(x: jax.Array):
    """Sigmoid Weighted Linear Unit activation function"""
    return x * jax.nn.sigmoid(x)

Model Architecture

In [ ]:
class energy_prediction(nnx.module):
    """Model architecture"""

    def __init__(self,dim_in: int, dim_hidden1_in: int, dim_hidden2_in: int,dim_hidden3_in, dim_out: int,*,rngs: nnx.Rngs):
        self.layer1 = Linear(din=dim_in,dout=dim_hidden1_in)
        self.layer2 = Linear(din=dim_hidden1_in,dout=dim_hidden2_in, rngs=rngs)
        self.layer3 = Linear(din=dim_hidden2_in,dout=dim_hidden3_in)
        self.layer4 = Linear(din=dim_hidden3_in,dout=dim_out)
        self.silu = SiLU()
        
    def __call__(self,x_in):
        # pass to calculate e
        def forwardPass(x):
            x = self.layer1(x)
            x = self.silu(x)
            x = self.layer2(x)
            x = self.silu(x)
            x = self.layer3(x)
            x = self.silu(x)
            x = self.layer4(x)
            return x.squeeze()
        
        e = forwardPass(x_in)

        # pass to calculate e_prime
        dedx = jax.grad(forwardPass,argnums=(0))
        e_prime = dedx(x_in)

        return e, e_prime


Define optimiser and loss

In [ ]:
optimiser = optax.adam(learning_rate=Learn_Rate, b1=beta_1, b2=beta_2)

def loss_fn(x: jax.Array, target_e, target_e_prime,*, model,alpha,gamma,lam): 
    """
    Calculates the loss of a model, works to minimise the mean square error of both 
    the strain energy prediction and the strain energy derivative prediction,
    whilst forcing the function through zero.
    """
    prediction_e, prediction_e_prime = model(x)
    loss_e = jnp.mean((prediction_e - target_e)**2)
    loss_e_prime = jnp.mean((prediction_e_prime - target_e_prime)**2)
    target_zero = 0
    x_zero = jnp.zeros(x[0].shape)
    x_zero = jnp.expand_dims(x_zero, axis=0)

    prediction_zero, _ = model(x_zero)
    loss_zero = jnp.mean((prediction_zero - target_zero)**2)

    return (alpha * loss_e + gamma * loss_e_prime + lam * loss_zero)

Train State Bundle

In [ ]:
class TrainState:
    def __init__(self,model,params,optimiser,opt_state,alpha,gamma,lambda_):
        self.model = model
        self.params = params
        self.optimiser = optimiser
        self.opt_state = opt_state
        self.alpha = alpha
        self.gamma = gamma
        self.lambda_ = lambda_

Train Step

In [ ]:
@nnx.jit
def training_step(TrainState,batch,loss_fn):
    Model = TrainState.model
    optimiser = TrainState.optimiser
    opt_state = TrainState.opt_state
    alpha = TrainState.alpha
    gamma = TrainState.gamma
    lambda_ = TrainState.lambda_

    disp_in = batch['displacements']
    e_target = batch['target_e']
    e_prime_target = batch['target_e_prime']

    def wrapped_loss_fn(Model):
        loss = loss_fn(
            disp_in,
            e_target,
            e_prime_target,
            Model,
            alpha=alpha,
            gamma=gamma,
            lam=lambda_
        )
        return loss

    loss = loss_fn(
        disp_in,
        e_target,
        e_prime_target,Model,
        alpha=alpha,gamma=gamma,
        lam=lambda_
    )
    grads = nnx.grad(wrapped_loss_fn)(Model)
    updates, new_opt_state = optimiser.update(grads.parameters(), opt_state, Model.parameters())
    nnx.apply_updates(Model.parameters(), updates)

    new_state = TrainState.replace(
        model=Model,
        opt_state=new_opt_state,
        params=TrainState.params,
        optimiser=optimiser,
        alpha=alpha,
        gamma=gamma,
        lambda_=lambda_
    )
    return new_state, loss

Training Loop

In [ ]:
# Instantiate energy prediction NN
Model = energy_prediction(
    dim_in=input_dataset.shape[1] * input_dataset.shape[2], 
    dim_hidden1_in=2024,
    dim_hidden2_in=1012,
    dim_hidden3_in=212, 
    dim_out=1,
    rngs=rngs
    )

params = nnx.get_parameters(Model) 
opt_state = optimiser.init(params)
train_state = TrainState(
    model=Model,
    params=params,
    optimiser=optimiser,
    opt_state=opt_state
    )

loss_record = []

for epoch in Epochs:
    running_loss = 0.0
    batch_count = 0

    for batch in tqdm(get_batches(Dataset, batch_size=Batch_size, shuffle=True),desc=f"Epoch {epoch}/{Epochs}", leave=False):
        new_state, loss_batch = training_step(TrainState,batch,loss_fn)

        running_loss += loss_batch
        batch_count += 1
    
    avg_loss = avg_loss = running_loss / batch_count if batch_count > 0 else 0.0
    loss_record.append(avg_loss)

Plots